# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
# print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
         # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list[:3])

# creating a smaller event data csv file called event_datafile_full csv 
# that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# define column list for event_datafile_full
columns_list = [
    'artist','firstName','gender','itemInSession','lastName',
    'length','level','location','sessionId','song','userId'
]

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(columns_list)
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
cql = """
CREATE KEYSPACE IF NOT EXISTS dw_sparkify 
WITH 
    REPLICATION = { 
        'class' : 'SimpleStrategy',
        'replication_factor' : 1 
    }
"""

try:
    session.execute(cql)
except Exception as e:
    print("Error: Could not able to create keyspace mydb due to: ")
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('dw_sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1

#### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

```sql
select 
    artist, 
    song, 
    length 
from dw_music_app_by_session 
where 
    sessionId = 338 
    and itemInSession = 4
```
In order to filter our table using `sessionId` and `itemInSession` columns, we need to define these as *Primary Key*.

> Table name: **dw_music_app_by_session**

|#|Column Name|Data Type|Primary Key|
|--:|:--|:--|--:|
|1|sessionId|int|1|
|2|itemInSession|int|1|
|3|artist|text||
|4|song|text||
|5|length|float||


#### Creating Target Table: **dw_music_app_by_session**

In [8]:
query = """
CREATE TABLE IF NOT EXISTS dw_music_app_by_session (
    sessionId int, 
    itemInSession int, 
    artist text, 
    song text, 
    length float, 
    PRIMARY KEY (sessionId, itemInSession)
)
"""

try:
    session.execute(query)
except Exception as e:
    print(e)             

#### Insert Data Into dw_music_app_by_session

In [9]:
filename = 'event_datafile_new.csv'

with open(filename, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO dw_music_app_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # line[8]: sessionId, line[3]: itemInSession, line[0]: artist, line[9]: song, line[5]: length
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into dw_music_app_by_session

In [10]:
cql = """
    select 
        artist, 
        song, 
        length 
    from dw_music_app_by_session 
    where 
        sessionId = 338 
        and itemInSession = 4
"""    

try:
    rows = session.execute(cql)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2

#### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

```sql
select 
    artist, 
    song, 
    user 
from dw_music_app_by_user 
where 
    userId = 10 
    and sessionId = 182
```
In order to filter our table using `userId` and `sessionId` columns also we want to order our data by `itemInSession`, we need to define `userId` and `sessionId` as *Primary Key* and `itemInSession` as *Clustering Key*.

> Table name: **dw_music_app_by_user**

|#|Column Name|Data Type|Primary Key|Clustering Key|
|--:|:--|:--|--:|--:|
|1|userId|int|1||
|2|sessionId|int|1||
|3|itemInSession|int|1|1|
|4|user|text||
|5|artist|text||
|6|song|text||


#### Creating Target Table: **dw_music_app_by_user**

In [11]:
query = """
CREATE TABLE IF NOT EXISTS dw_music_app_by_user (
    userId int, 
    sessionId int, 
    itemInSession int,
    user text,
    artist text, 
    song text, 
    PRIMARY KEY ((userId, sessionId), itemInSession)
)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)           

#### Insert Data into **dw_music_app_by_user**

In [12]:
filename = 'event_datafile_new.csv'

with open(filename, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO dw_music_app_by_user (userId, sessionId, itemInSession, user, artist, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        # line[10]: userId, line[8]: sessionId, line[3]: itemInSession, line[1]: firstName line[4]: lastName,
        # line[0]: artist, line[9]: song
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), f'{line[1]} {line[4]}', line[0], line[9]))

#### Do a SELECT to verify that the data have been inserted into dw_music_app_by_user

In [13]:
cql = """
    select 
        artist, 
        song, 
        user 
    from dw_music_app_by_user 
    where 
        userId = 10 
        and sessionId = 182
"""    

try:
    rows = session.execute(cql)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3

#### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

```sql
select 
    user 
from dw_music_app_by_user_song 
where 
    song = 'All Hands Against His Own'
```
In order to filter our table using `song` column. But we want to be able to get every user who listened a specific song, this is why we will use a composite key as primary key, which means `user` column will be our second column to built our primary key.

> Table name: **dw_music_app_by_user_song**

|#|Column Name|Data Type|Primary Key|
|--:|:--|:--|--:|
|1|song|text|1|
|2|user|text|1|

#### Creating Target Table: **dw_music_app_by_user_song**

In [14]:
query = """
    CREATE TABLE IF NOT EXISTS dw_music_app_by_user_song (
        song text, 
        user text, 
        PRIMARY KEY (song, user)
    )
"""
try:
    session.execute(query)
except Exception as e:
    print(e)   

                    

#### Insert Data into **dw_music_app_by_user_song**

In [15]:
filename = 'event_datafile_new.csv'

with open(filename, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO dw_music_app_by_user_song (song, user)"
        query = query + "VALUES (%s, %s)"
        # line[9]: song, line[1]: firstName line[4]: lastName
        session.execute(query, (line[9], f'{line[1]} {line[4]}'))

#### Do a SELECT to verify that the data have been inserted into dw_music_app_by_user_song

In [16]:
cql = """
    select 
        user 
    from dw_music_app_by_user_song 
    where 
        song = 'All Hands Against His Own'
"""    

try:
    rows = session.execute(cql)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user)

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [17]:
for t in ["dw_music_app_by_session", "dw_music_app_by_user", "dw_music_app_by_user_song"]:
    query = f"DROP TABLE {t}"
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()

---